In [ ]:
from llama_index.core import (
    Document,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline
from llama_index.vector_stores.chroma import ChromaVectorStore

import chromadb
import nest_asyncio

nest_asyncio.apply()

In [ ]:
text = "Hello everyone! This is a Mental Assistant project for Senior Design. Nice to meet you all!"
doc = Document(text=text)
index = VectorStoreIndex.from_documents([doc])

In [ ]:
print(index)

In [ ]:
index.vector_store

In [ ]:
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=512, chunk_overlap=20),
        TitleExtractor(),
        OpenAIEmbedding(),
    ]
)

nodes = pipeline.run(documents=[doc])

In [ ]:
index = VectorStoreIndex(nodes)

In [ ]:
index

In [ ]:
index.vector_store

In [ ]:
index.storage_context.persist(persist_dir="./index_cache")
storage_context = StorageContext.from_defaults(persist_dir="./index_cache")
reloaded_index = load_index_from_storage(storage_context)

## Chromadb

In [ ]:
db = chromadb.PersistentClient(path="./index_cache/database")
collections = db.get_or_create_collection("my_collection")

vector_store = ChromaVectorStore(chroma_collection=collections)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents=[doc],
    storage_context=storage_context,
)